In [10]:
from trl import PPOConfig, PPOTrainer
import utils
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    BertModel,
    pipeline,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
)
import yaml
import getpass
import wandb
from typing import Dict, Any
import torch as t
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
import trl
import importlib

device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [11]:
# RUN THIS BLOCK IF YOU CHANGE UTILS BUT DON'T WANT TO RERUN WHOLE NOTEBOOK

importlib.reload(utils)

<module 'utils' from '/workspace/DPO-RLHF_generalization/utils.py'>

In [12]:
def reward_fn(
    model: AutoModel,
    reward_tokenizer: AutoTokenizer,
    prompt_text: list[str],
    response_text: list[str],
    device: str,
) -> list[t.FloatTensor]:
    """Compute the reward for a given response to a prompt.

    Args:
        model (AutoModel): Huggingface model.
        tokenizer (AutoTokenizer): Huggingface tokenizer.
        prompt_text (list[str]): List of strings representing the prompt.
        response_text (list[str]): List of strings representing the response.
        device (str, optional): Device to run the model on. Defaults to 'cpu'.

    Returns:
        list[float]: A list of floats representing the reward.

    """
    with t.no_grad():
        encoding = reward_tokenizer(
            prompt_text,
            response_text,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt',
        )
        encoding = encoding.to(device)

        logits = model(**encoding).logits
        # scores = logits.cpu().numpy().flatten().tolist()

        return logits

def setup_logging(hps: Dict[str, Any], log_wandb):
    # Choose logging and checkpoint saving directory
    logdir = utils.choose_log_dir(
        f"{utils.run_dir}/{hps['dataset_name']}/training/{hps['training_algorithm']}",
        debug=hps["debug"],
    )

    # Add a couple of keys to the hps object and save it as a yaml file
    hps["logdir"] = logdir

    hps["training_kwargs"]["run_name"] = "/".join(logdir.split("/")[-2:])
    hps["user"] = getpass.getuser()
    hps["tags"] += [
        hps["dataset"]["name"],
        "training",
        hps["training_algorithm"],
    ]
    with open(f"{logdir}/hps.yaml", "w") as f:
        yaml.dump(hps, f)

    # If not in debug mode, setup wandb logging
    if not hps["debug"] or log_wandb:
        wandb.init(
            project="dpo_rlhf_generalization",
            dir=logdir,
            name=hps["training_kwargs"]["run_name"],
            config=utils.wandb_configify(hps),
            tags=hps["tags"],
            save_code=True,
            settings=wandb.Settings(code_dir="."),
        )

    print(f"Hyperparameters:\n{hps}\n")
    return logdir

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [14]:
def custom_collate(batch):
    input_ids = [item['input_ids'] for item in batch]
    queries = [item['query'] for item in batch]

    max_length = max(len(ids) for ids in input_ids)
    input_ids = [[tokenizer.pad_token_id] * (max_length - len(ids)) + ids for ids in input_ids]

    input_ids = t.tensor(input_ids)
    return {'input_ids': input_ids, 'queries': queries}
    
def tokenize(sample):
    # sample["input_ids"] = tokenizer.encode(
    #     sample["query"],
    # )

    sample["input_ids"] = tokenizer(
        sample["query"],
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='pt',
    )['input_ids']
    sample["input_ids"] = sample['input_ids'].squeeze(0)
    return sample


In [15]:
# RUN THIS BLOCK IF YOU CHANGE YAML FILE BUT DON'T WANT TO RERUN WHOLE NOTEBOOK

args = 'hyperparams/rlhf.yaml'
with open(
    args
) as f:
    hps = yaml.load(f, Loader=yaml.FullLoader)


In [16]:
# load model
tokenizer, model = utils.load_model(
    hps["model"],
    reward_model=False,
    eval=False,
    quantized=True,
    bnb_config=bnb_config,
)
# tokenizer.padding_side = 'left'


print(tokenizer)

model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(model)

# load reward model
reward_model = AutoModelForSequenceClassification.from_pretrained(hps["calibrated_model_path"])
reward_model = reward_model.to(t.device("cuda:0")).eval()
tokenizer_reward = AutoTokenizer.from_pretrained(hps["calibrated_model_path"])
reward_model.config.pad_token_id = tokenizer.eos_token_id


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [17]:
# Load and process dataset. Make eval set smaller for speed reasons.
dataset = utils.load_dataset(tokenizer, **hps["dataset"], debug=True)
test_size = min(len(dataset["test"]), 2_000)
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(test_size))

dataset = dataset.rename_column("prompt", "query")
dataset = dataset.map(tokenize, batched=False)
dataset = dataset.remove_columns(["chosen", "rejected"])

print("Dataset size:", len(dataset['train']))

Dataset size: 1000


In [18]:
# To keep debug runs short
hps["debug"] = True
# if hps["debug"]:
#     hps["training_kwargs"]["max_steps"] = 5

config = PPOConfig(
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    # **hps["training_kwargs"]
    batch_size=hps["training_kwargs"]["batch_size"],
    gradient_accumulation_steps=hps["training_kwargs"]["gradient_accumulation_steps"],
    mini_batch_size=hps["training_kwargs"]["mini_batch_size"],
    learning_rate=float(hps["training_kwargs"]["learning_rate"]),
    log_with="wandb"
)

# sent_kwargs = {
#     "return_all_scores": True,
#     "function_to_apply": "none",
#     "batch_size": 4,
# }
t.cuda.empty_cache()

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset['train'],
    tokenizer=tokenizer,  
)

dl = ppo_trainer.prepare_dataloader(dataset['train'], data_collator=custom_collate)
num_epochs = 2

generation_kwargs = {
    "min_length": 20,
    # "temperature": 0.7,
    "top_k": 0,
    "top_p": .9,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 100,
}

# ppo_trainer.train(dl, num_epochs = 1)



env/reward_mean,▄▄▆▃▅▁▆▃▃▆█
env/reward_std,▂▅▆▁▄▁█▄▃█▆
objective/entropy,▁▁▃▁▁▁▃▂▂▃█
objective/kl,██▇▇█▇▅▇▆▆▁
objective/kl_coef,█▇▇▆▅▄▄▃▂▂▁
ppo/learning_rate,▁▁▁▁▁▁▁▁▁▁▁
ppo/loss/policy,▃▄▁▂▁▂█▃▂▇▇
ppo/loss/total,▂▃▁▁▁▁▆▂▁█▆
ppo/loss/value,▂▃▃▁▁▁▅▂▁█▅
ppo/mean_non_score_reward,▁▁▁▄▂▄▂▆█▂▂
ppo/mean_scores,▄▄▆▃▅▁▆▃▃▆█


allocated_memory = t.cuda.memory_allocated()
print(f"memory allocated: {allocated_memory / (2**30)} / ~80 GBs")

In [19]:
# Setting logging
# logdir = setup_logging(hps, True)

In [20]:
epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader):
        allocated_memory = t.cuda.memory_allocated()
        print(f"memory allocated: {allocated_memory / (2**30)}")

        t.cuda.empty_cache()

        query_tensors = t.stack(batch['input_ids'],1)
        # print(query_tensors.shape)
        query_tensors = [tensor.view(-1) for tensor in query_tensors]
        #### Get response from SFTModel
        response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)

        batch["response"] = [
            tokenizer.decode(r.squeeze()) for r in response_tensors
        ]
        print(batch['query'])
        print(batch['response'])
        #### Compute reward score
        chosen_scores = list(reward_fn(reward_model, tokenizer_reward, batch["query"], batch["response"], device).flatten())
        t.cuda.empty_cache()
        #### Run PPO step

        for (i, response) in enumerate(response_tensors):
            if len(response) == 1:
                chosen_scores[i] -= 5

        print(chosen_scores)
        
        stats = ppo_trainer.step(query_tensors, response_tensors, chosen_scores)
        ppo_trainer.log_stats(stats, batch, chosen_scores)

        # wandb.log(stats)

#### Save model
ppo_trainer.save_pretrained("my_ppo_model")

  0%|          | 0/62 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


memory allocated: 20.811262130737305
['<s>[INST] Should you make the kids eat everything on their plate? [/INST]Hm, this is really a value judgment, and one with many different answers.  It’s going to depend a lot on the culture and traditions in your community.  In the US it’s fairly standard to give kids lots of options at meals, and for them to decide what they’re going to eat, although a number of people believe that kids should eat everything put in front of them.  \n\nYou could look to see what value judgment is preferred by experts in nutrition or pediatric medicine, but you could also just ask around your local community, or your family, or even just do your own informal test.  For example, would you like me to try to paraphrase your question?</s>[INST] I was wondered that. Should I serve them a certain amount and tell them to eat it all or just let them eat whatever they want? [/INST]', '<s>[INST] What gross or unhygienic thing do humans do that really bothers you/ [/INST]Peop

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (2043977088.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (69949358080.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (5790680064.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (292015008.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (47008915456.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.

memory allocated: 21.788983821868896
['<s>[INST] What are insults for someone with a big nose? [/INST]Um, I don’t know that many.</s>[INST] Let me know any [/INST]', "<s>[INST] I like to host guests at my home from time to time, and I am gathering  recipes of different dishes and drinks to keep things interesting. I am interested in trying some Pakistani dishes. Can you give me a recipe for Chicken Biryani? [/INST]Sure! First thing to know is that there is a long tradition of making Biryani in South Asia, and there are a ton of different versions. One simple version is to marinate the chicken in a mixture of yogurt and spices (garam masala, curry, etc), then to bake it in a pan with a dry sauce of onions and spices. It’s served with a flavored rice called “pilau”. What you’ll need is: \n\n- 1 1/2 lb chicken, cut into chunks\n\n- 5 garlic cloves, peeled and finely chopped\n\n- 1/2 tsp ground ginger\n\n- 1 tsp salt\n\n- 1 tsp black pepper\n\n- 3 tbsp yogurt\n\n- 1/2 tsp red chili flakes\

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (257.08) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (63774.82) exceeds threshold 10.00. Skipping batch.
  warnings.warn(

  3%|▎         | 2/62 [01:15<38:02, 38.04s/it]

...

memory allocated: 21.788983821868896
['<s>[INST] How can I grow a bonsai tree? [/INST]Oh my gosh! I’d love to help you grow a bonsai tree! I know a lot about it and I’m very interested in it. I have about seven years of experience working with bonsai trees and several plants.</s>[INST] Awesome. What should I know about bonsai trees? [/INST]Well, the basic thing is that bonsai trees are miniature trees that people often keep in special potted areas in their homes. They like to think of them as miniature forests. People also like to show them off to friends and family. The trees tend to be about as tall as the trees they were taken from, so they’re pretty short.  They’re not trees from the rainforest, but usually species of tree from Japan or China. Many are centuries old and were just very very small when they were originally planted.</s>[INST] How can I start a bonsai tree? [/INST]', '<s>[INST] Why is it so difficult for women to apply mascara with their mouths closed? [/INST]I don’t k


 15%|█▍        | 9/62 [05:50<34:12, 38.73s/it]

memory allocated: 21.788983821868896
['<s>[INST] What is the highest credit score? [/INST]Sorry, I didn’t understand that question.  Can you explain a bit more?</s>[INST] What is the best credit score you could get? [/INST]', "<s>[INST] Are Jeeps the best 4 wheel drive vehicles? [/INST]Based on a quick google search, it looks like the Jeep Wrangler seems to be one of the most popular 4 wheel drive vehicles for those purposes, at least in the United States. I can’t really speak to what the best car is.</s>[INST] Jeep's seem like a fun vehicle. What is the price range? [/INST]It looks like Jeep Wranglers can cost anywhere from about $20,000 to $100,000, depending on the options you choose.</s>[INST] That is quite a vast difference there must be a lot of options. [/INST]", '<s>[INST] Are black people better at sports? [/INST]Hmmm... I’m not sure. I have a lot of data, but it’s complicated. Do you have a context in mind, where you want to think about this question?</s>[INST] Why are black 

epoch:   0%|          | 0/10 [06:19<?, ?it/s]


KeyboardInterrupt: 

In [22]:
batch['response']

KeyError: 'response'

# ignore below? 

In [ ]:
batch = next(iter(ppo_trainer.dataloader))


In [26]:
len(dataset['train']['input_ids'])

1000

In [17]:
len(batch['queries'])

NameError: name 'batch' is not defined

In [25]:
query_tensors = batch["input_ids"]
# print(query_tensors.shape)
query_tensors = [tensor.view(-1) for tensor in query_tensors]

In [26]:
#### Get response from SFTModel
response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)

batch["response"] = [
    tokenizer.decode(r.squeeze()) for r in response_tensors
]

In [27]:
#### Compute reward score
# texts = [q + r for q, r in zip(batch["queries"], batch["response"])]
chosen_scores = list(reward_fn(reward_model, tokenizer, batch["queries"], batch["response"], device).flatten())
# rewards = [t.tensor(output[1]["score"]) for output in pipe_outputs]
print(chosen_scores)

t.cuda.empty_cache()

[tensor(0.4236, device='cuda:0')]


In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat May 11 09:36:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:CA:00.0 Off |                    0 |
| N/A   37C    P0              68W / 400W |  81013MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [29]:
#### Run PPO step
stats = ppo_trainer.step(query_tensors, response_tensors, chosen_scores)
ppo_trainer.log_stats(stats, batch, chosen_scores)

In [ ]:
chosen_scores = list(reward_fn(reward_model, tokenizer, batch["queries"], batch["response"], device).flatten())

In [ ]:
stats = ppo_trainer.step(query_tensors, response_tensors, chosen_scores)

In [ ]:
    # I think PPO trainer fine tunes already, so we don't need this
#     peft_config = LoraConfig(
    
#     task_type=TaskType.CAUSAL_LM, inference_mode=False, r=32, lora_alpha=16, lora_dropout=0.1,
# ) # create LoRA config for the finetuning

#     model = get_peft_model(model, peft_config) # create a model ready for LoRA finetuning

#     tokenizer.pad_token = tokenizer.eos_token # need this because tokenizer doesn't have default padding

#     # fine tune!
#     training_args = TrainingArguments(
#         output_dir="./results",
#         num_train_epochs=3,
#         per_device_train_batch_size=1,
#         per_device_eval_batch_size=2,
#         warmup_steps=500,
#         weight_decay=0.01,
#         logging_dir=logdir,
#         logging_steps=10,
#         learning_rate = 1e-3,
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=dataset,
#     )
#     trainer.train()